In [1]:
import datetime as dt
import matplotlib.pyplot as plt
from matplotlib import style
import pandas as pd
import pandas_datareader.data as web
import urllib as u
import urllib.request
from bs4 import BeautifulSoup as bs
import datetime as dt
import numpy as np
import os
import pickle
import requests
import yfinance as yf

In [2]:
def get_price2book( symbol ):
    try:
        url = r'http://finviz.com/quote.ashx?t={}'\
        				.format(symbol.lower())
        html = u.request.urlopen(url).read()
        soup = bs(html, 'lxml')
        # Change the text below to get a diff metric
        pb =  soup.find(text = r'P/B')
        pb_ = pb.find_next(class_='snapshot-td2').text
        roe =  soup.find(text = r'ROE')
        roe_ = roe.find_next(class_='snapshot-td2').text
        mc =  soup.find(text = r'Market Cap')
        mc_ = mc.find_next(class_='snapshot-td2').text
        de =  soup.find(text = r'Debt/Eq')
        de_ = de.find_next(class_='snapshot-td2').text
        dp =  soup.find(text = r'Dividend %')
        dp_ = dp.find_next(class_='snapshot-td2').text
        #print( '{} price to book = {} '.format(symbol, pb_) )
        #print( '{} roe = {} '.format(symbol, roe_) )
        return (pb_,roe_,mc_,de_,dp_)
    except Exception as e:
        print(e)

In [3]:
style.use('ggplot')


def save_sp500_tickers():
    resp = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    soup = bs(resp.text, 'lxml')
    table = soup.find('table', {'class': 'wikitable sortable'})
    tickers = []
    for row in table.findAll('tr')[1:]:
        ticker = row.findAll('td')[0].text
        tickers.append(ticker)
    with open("sp500tickers.pickle", "wb") as f:
        pickle.dump(tickers, f)
    return tickers


#save_sp500_tickers()

In [4]:
def get_data_from_yahoo(reload_sp500=False):
    if reload_sp500:
        tickers = save_sp500_tickers()
    else:
        with open("sp500tickers.pickle", "rb") as f:
            tickers = pickle.load(f)
    if not os.path.exists('stock_dfs'):
        os.makedirs('stock_dfs')

    start = dt.datetime(2010, 1, 1)
    end = dt.datetime.now()
    for ticker in tickers:
        print(ticker.rstrip())
        print(format(ticker))
        # just in case your connection breaks, we'd like to save our progress!
        if not os.path.exists('stock_dfs/{}.csv'.format(ticker.splitlines()[0])):
            df = yf.download(ticker, start, end)
            #df.reindex(columns=cols)
            df.reset_index(inplace=True)
            df.set_index("Date", inplace=True)
            df = df.drop("Adj Close", axis=1)
            print(df)
            df.to_csv('stock_dfs/{}.csv'.format(ticker.splitlines()[0]))
        else:
            print('Already have {}'.format(ticker.splitlines()[0]))

#get_data_from_yahoo()
            
def compile_data():
    with open("sp500tickers.pickle", "rb") as f:
        tickers = pickle.load(f)

    main_df = pd.DataFrame()

    for count, ticker in enumerate(tickers):
        df = pd.read_csv('stock_dfs/{}.csv'.format(ticker.splitlines()[0]))
        df.set_index('Date', inplace=True)

        df.rename(columns={'Adj Close': ticker}, inplace=True)
        df.drop(['Open', 'High', 'Low', 'Close', 'Volume'], 1, inplace=True)

        if main_df.empty:
            main_df = df
        else:
            main_df = main_df.join(df, how='outer')

        if count % 10 == 0:
            print(count)
    print(main_df.head())
    main_df.to_csv('sp500_joined_closes.csv')


def visualize_data():
    df = pd.read_csv('sp500_joined_closes.csv')
    df_corr = df.corr()
    print(df_corr.head())
    df_corr.to_csv('sp500corr.csv')
    data1 = df_corr.values
    fig1 = plt.figure()
    ax1 = fig1.add_subplot(111)

    heatmap1 = ax1.pcolor(data1, cmap=plt.cm.RdYlGn)
    fig1.colorbar(heatmap1)

    ax1.set_xticks(np.arange(data1.shape[1]) + 0.5, minor=False)
    ax1.set_yticks(np.arange(data1.shape[0]) + 0.5, minor=False)
    ax1.invert_yaxis()
    ax1.xaxis.tick_top()
    column_labels = df_corr.columns
    row_labels = df_corr.index
    ax1.set_xticklabels(column_labels)
    ax1.set_yticklabels(row_labels)
    plt.xticks(rotation=90)
    heatmap1.set_clim(-1, 1)
    plt.tight_layout()
    plt.show()



#visualize_data()

In [5]:
def analyse_data_from_yahoo():
    with open("sp500tickers.pickle", "rb") as f:
        tickers = pickle.load(f)
    analysis = pd.DataFrame(columns=['drop ratio','pb','roe','market cap','price on 04-09','Debt/Eq','Dividend %'])
    
    i=0
    for ticker in tickers:
        i=i+1
        #print(ticker.splitlines()[0])
        #print(df.shape)
        ticker = ticker[:-1]
        if os.path.exists('stock_dfs/{}.csv'.format(ticker)):
            df = pd.read_csv('stock_dfs/{}.csv'.format(ticker), parse_dates=True, index_col=0)
            print(ticker,df.shape,df.shape[0] )
            df['10ma']=(df['Close'].rolling(window=10, min_periods=0).mean())
            #print(df['10ma'])
            #analysis.loc[i] =  [ticker  ,  df['10ma'].iloc[320]/df['10ma'].iloc[1] ]
            #analysis[ticker]  = [  df['10ma'].iloc[320]/df['10ma'].iloc[1] ]
            if df.shape[0]==2587:
                analysis.loc[ticker,'drop ratio']  = df.loc['2020-04-09','10ma']/df.loc['2020-01-06','10ma'] 
                analysis.loc[ticker,'price on 04-09']  =  df.loc['2020-04-09','Close'] 
            print(ticker,i)
            try:
                temp= get_price2book(ticker)    
                analysis.loc[ticker,'pb']  = temp[0]
                analysis.loc[ticker,'roe']  = temp[1]
                analysis.loc[ticker,'market cap']  = temp[2]
                analysis.loc[ticker,'Debt/Eq']  = temp[3]
                analysis.loc[ticker,'Dividend %']  = temp[4]
                print(analysis.loc[ticker])
            except:
                print("An exception occurred")
        else:
            print('Already have {}'.format(ticker))
    
    analysis.to_csv("500_analysis.csv")

In [6]:
analyse_data_from_yahoo()

MMM (2587, 5) 2587
MMM 1
drop ratio        0.786158
pb                    8.60
roe                 45.10%
market cap          87.08B
price on 04-09      147.78
Debt/Eq               2.03
Dividend %           3.92%
Name: MMM, dtype: object
ABT (2587, 5) 2587
ABT 2
drop ratio        0.925312
pb                    5.07
roe                 11.70%
market cap         157.60B
price on 04-09       86.04
Debt/Eq               0.58
Dividend %           1.62%
Name: ABT, dtype: object
ABBV (1833, 5) 1833
ABBV 3
drop ratio            NaN
pb                      -
roe               -95.60%
market cap        121.88B
price on 04-09        NaN
Debt/Eq                 -
Dividend %          5.75%
Name: ABBV, dtype: object
ABMD (2587, 5) 2587
ABMD 4
drop ratio        0.876052
pb                    6.96
roe                 24.70%
market cap           7.28B
price on 04-09      160.08
Debt/Eq               0.00
Dividend %               -
Name: ABMD, dtype: object
ACN (2588, 5) 2588
ACN 5
drop ratio          

drop ratio        0.991218
pb                    3.95
roe                 10.20%
market cap          24.18B
price on 04-09      131.75
Debt/Eq               1.55
Dividend %           1.50%
Name: AWK, dtype: object
AMP (2587, 5) 2587
AMP 37
drop ratio        0.616349
pb                    2.57
roe                 32.10%
market cap          14.74B
price on 04-09      120.47
Debt/Eq               3.38
Dividend %           3.39%
Name: AMP, dtype: object
ABC (2587, 5) 2587
ABC 38
drop ratio        1.01157
pb                   6.24
roe                22.10%
market cap         18.52B
price on 04-09      87.81
Debt/Eq              1.41
Dividend %          1.88%
Name: ABC, dtype: object
AME (2587, 5) 2587
AME 39
drop ratio        0.734071
pb                    3.49
roe                 18.00%
market cap          17.85B
price on 04-09       78.23
Debt/Eq               0.54
Dividend %           0.92%
Name: AME, dtype: object
AMGN (2587, 5) 2587
AMGN 40
drop ratio        0.86206
pb                 

drop ratio        0.875349
pb                    2.12
roe                 13.90%
market cap          71.08B
price on 04-09      471.42
Debt/Eq               0.15
Dividend %           3.17%
Name: BLK, dtype: object
BA (2587, 5) 2587
BA 73
drop ratio        0.43224
pb                      -
roe                14.30%
market cap         79.98B
price on 04-09     151.84
Debt/Eq                 -
Dividend %              -
Name: BA, dtype: object
BKNG (2587, 5) 2587
BKNG 74
drop ratio        0.642764
pb                   10.19
roe                 80.70%
market cap          60.45B
price on 04-09     1420.64
Debt/Eq               1.45
Dividend %               -
Name: BKNG, dtype: object
BWA (2587, 5) 2587
BWA 75
drop ratio        0.550386
pb                    1.19
roe                 16.70%
market cap           5.57B
price on 04-09       25.74
Debt/Eq               0.42
Dividend %           2.50%
Name: BWA, dtype: object
BXP (2587, 5) 2587
BXP 76
drop ratio        0.672628
pb                  

drop ratio        0.851028
pb                    1.52
roe                 11.60%
market cap          70.73B
price on 04-09      181.83
Debt/Eq               0.83
Dividend %           0.02%
Name: CI, dtype: object
CINF (2587, 5) 2587
CINF 109
drop ratio        0.737024
pb                    1.44
roe                 21.60%
market cap          13.71B
price on 04-09       83.58
Debt/Eq               0.09
Dividend %           2.76%
Name: CINF, dtype: object
CTAS (2587, 5) 2587
CTAS 110
drop ratio        0.661587
pb                    6.32
roe                 31.20%
market cap          20.32B
price on 04-09      198.74
Debt/Eq               0.82
Dividend %           1.26%
Name: CTAS, dtype: object
CSCO (2587, 5) 2587
CSCO 111
drop ratio        0.837567
pb                    5.11
roe                 31.50%
market cap         183.80B
price on 04-09        41.2
Debt/Eq               0.45
Dividend %           3.27%
Name: CSCO, dtype: object
C (2587, 5) 2587
C 112
drop ratio        0.527291
pb   

drop ratio        0.421474
pb                    1.03
roe                 33.30%
market cap          15.75B
price on 04-09       24.39
Debt/Eq               0.73
Dividend %               -
Name: DAL, dtype: object
XRAY (2587, 5) 2587
XRAY 144
drop ratio        0.675772
pb                    1.78
roe                  5.10%
market cap           9.04B
price on 04-09       41.87
Debt/Eq               0.28
Dividend %           0.98%
Name: XRAY, dtype: object
DVN (2587, 5) 2587
DVN 145
drop ratio        0.304079
pb                    0.59
roe                 -5.30%
market cap           3.39B
price on 04-09        9.67
Debt/Eq               0.78
Dividend %           4.83%
Name: DVN, dtype: object
FANG (1886, 5) 1886
FANG 146
drop ratio          NaN
pb                 0.39
roe               1.70%
market cap        5.39B
price on 04-09      NaN
Debt/Eq            0.41
Dividend %        4.74%
Name: FANG, dtype: object
DLR (2587, 5) 2587
DLR 147
drop ratio        1.17685
pb                   3.58

...

drop ratio        0.65243
pb                   1.12
roe                12.30%
market cap         11.65B
price on 04-09      39.43
Debt/Eq              0.42
Dividend %          4.19%
Name: NUE, dtype: object
NVDA (2587, 5) 2587
NVDA 355
drop ratio         1.0877
pb                  14.17
roe                25.70%
market cap        176.42B
price on 04-09     262.95
Debt/Eq              0.00
Dividend %          0.23%
Name: NVDA, dtype: object
NVR (2587, 5) 2587
NVR 356
drop ratio        0.699014
pb                    4.55
roe                 40.20%
market cap          10.73B
price on 04-09     2928.01
Debt/Eq               0.26
Dividend %               -
Name: NVR, dtype: object
ORLY (2587, 5) 2587
ORLY 357
drop ratio        0.723732
pb                   67.88
roe                514.90%
market cap          26.27B
price on 04-09      342.44
Debt/Eq               9.79
Dividend %               -
Name: ORLY, dtype: object
OXY (2587, 5) 2587
OXY 358
drop ratio        0.312835
pb               

drop ratio        0.936573
pb                    7.15
roe                 25.40%
market cap          34.15B
price on 04-09      208.59
Debt/Eq               0.38
Dividend %           3.90%
Name: PSA, dtype: object
PHM (2587, 5) 2587
PHM 390
drop ratio        0.581208
pb                    1.27
roe                 19.50%
market cap           6.95B
price on 04-09       26.34
Debt/Eq               0.57
Dividend %           1.87%
Name: PHM, dtype: object
PVH (2587, 5) 2587
PVH 391
drop ratio        0.375471
pb                    0.60
roe                  7.10%
market cap           3.50B
price on 04-09       49.51
Debt/Eq               0.48
Dividend %               -
Name: PVH, dtype: object
QRVO (1329, 5) 1329
QRVO 392
drop ratio           NaN
pb                  2.40
roe                8.00%
market cap        10.88B
price on 04-09       NaN
Debt/Eq             0.36
Dividend %             -
Name: QRVO, dtype: object
PWR (2587, 5) 2587
PWR 393
drop ratio        0.773851
pb                  

drop ratio        0.870345
pb                    2.26
roe                 17.50%
market cap          62.25B
price on 04-09       60.33
Debt/Eq               1.70
Dividend %           4.20%
Name: SO, dtype: object
LUV (2587, 5) 2587
LUV 425
drop ratio        0.614957
pb                    1.86
roe                 23.30%
market cap          18.29B
price on 04-09       36.47
Debt/Eq               0.27
Dividend %           2.07%
Name: LUV, dtype: object
SWK (2587, 5) 2587
SWK 426
drop ratio        0.618499
pb                    2.26
roe                 12.80%
market cap          17.05B
price on 04-09      119.37
Debt/Eq               0.42
Dividend %           2.39%
Name: SWK, dtype: object
SBUX (2587, 5) 2587
SBUX 427
drop ratio        0.761598
pb                       -
roe                -66.70%
market cap          86.96B
price on 04-09       73.88
Debt/Eq                  -
Dividend %           2.23%
Name: SBUX, dtype: object
STT (2587, 5) 2587
STT 428
drop ratio        0.676198
pb     

drop ratio         NaN
pb                1.97
roe                  -
market cap           -
price on 04-09     NaN
Debt/Eq              -
Dividend %           -
Name: UA, dtype: object
UNP (2587, 5) 2587
UNP 460
drop ratio        0.79151
pb                   5.69
roe                32.90%
market cap        101.69B
price on 04-09     148.73
Debt/Eq              1.39
Dividend %          2.60%
Name: UNP, dtype: object
UAL (2587, 5) 2587
UAL 461
drop ratio        0.308967
pb                    0.68
roe                 28.00%
market cap           7.79B
price on 04-09        31.5
Debt/Eq               1.29
Dividend %               -
Name: UAL, dtype: object
UNH (2587, 5) 2587
UNH 462
drop ratio        0.844457
pb                    4.45
roe                 25.50%
market cap         260.18B
price on 04-09      264.13
Debt/Eq               0.71
Dividend %           1.60%
Name: UNH, dtype: object
UPS (2587, 5) 2587
UPS 463
drop ratio        0.806457
pb                   26.75
roe               

drop ratio        0.607143
pb                    1.58
roe                  6.10%
market cap          20.99B
price on 04-09       16.59
Debt/Eq               1.67
Dividend %           9.24%
Name: WMB, dtype: object
WLTW (2587, 5) 2587
WLTW 495
drop ratio        0.848313
pb                    2.50
roe                 10.40%
market cap          25.61B
price on 04-09      196.02
Debt/Eq               0.55
Dividend %           1.38%
Name: WLTW, dtype: object
WYNN (2587, 5) 2587
WYNN 496
drop ratio        0.425573
pb                    4.56
roe                  6.30%
market cap           7.83B
price on 04-09       70.65
Debt/Eq               5.98
Dividend %           5.37%
Name: WYNN, dtype: object
XEL (2587, 5) 2587
XEL 497
drop ratio        0.963625
pb                    2.63
roe                 10.70%
market cap          34.78B
price on 04-09       66.27
Debt/Eq               1.42
Dividend %           2.45%
Name: XEL, dtype: object
XRX (2587, 5) 2587
XRX 498
drop ratio        0.495986
pb 

In [55]:
df_new = pd.read_csv('500_analysis.csv',index_col=0)

In [64]:
print(df_new)

      drop ratio         pb          roe   market cap  price on 04-09  \
MMM          NaN   ['8.60']   ['45.10%']   ['87.08B']             NaN   
ABT          NaN   ['5.07']   ['11.70%']  ['157.60B']             NaN   
ABBV         NaN      ['-']  ['-95.60%']  ['121.88B']             NaN   
ABMD         NaN   ['6.96']   ['24.70%']    ['7.28B']             NaN   
ACN          NaN   ['7.34']   ['33.80%']  ['113.77B']             NaN   
...          ...        ...          ...          ...             ...   
YUM          NaN      ['-']  ['-16.20%']   ['24.21B']             NaN   
ZBRA         NaN   ['6.07']   ['33.20%']   ['11.17B']             NaN   
ZBH          NaN   ['1.89']    ['9.50%']   ['23.67B']             NaN   
ZION         NaN   ['0.73']   ['11.20%']    ['4.93B']             NaN   
ZTS          NaN  ['22.39']   ['59.30%']   ['60.60B']             NaN   

       Debt/Eq Dividend %  
MMM   ['2.03']  ['3.92%']  
ABT   ['0.58']  ['1.62%']  
ABBV     ['-']  ['5.75%']  
ABMD  ['0.0